# Предобработка данных и функции потерь в линейной регрессии

## Данные
Мы будем работать с набором данных [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/Automobile). В данных присутствуют категориальные, целочисленные и вещественнозначные признаки.

In [86]:
import numpy as np
import pandas as pd

X_raw = pd.read_csv("auto.csv", header=None, na_values=["?"])

X_raw.columns=range(1,27)

X_raw.head(20)

,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,25,26
0,3,NaN,0,1,0,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,0,1,0,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,0,1,0,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,1,1,0,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,1,1,0,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
5,2,NaN,1,1,0,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110.0,5500.0,19,25,15250.0
6,1,158.0,1,1,0,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110.0,5500.0,19,25,17710.0
7,1,NaN,1,1,0,four,wagon,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110.0,5500.0,19,25,18920.0
8,1,158.0,1,1,1,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.40,8.3,140.0,5500.0,17,20,23875.0
9,0,NaN,1,1,1,two,hatchback,4wd,front,99.5,...,131,mpfi,3.13,3.40,7.0,160.0,5500.0,16,22,NaN


## Предобработка данных
Предобработка данных важна при применении любых методов машинного обучения, а в особенности для линейных моделей. В sklearn предобработку удобно делать с помощью модуля [preprocessing](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)

In [20]:
from sklearn import preprocessing

### Преобразование нечисловых признаков
Практически все методы машинного обучения требуют, чтобы на вход функции обучения подавалась вещественная матрица. В процессе обучения используются свойства вещественных чисел, в частности, возможность сравнения и применения арифметических операций. Поэтому, даже если формально в матрице объекты-признаки записаны числовые значения, нужно всегда анализировать, можно ли относиться к ним как к числам. 

__Пример:__ некоторые признаки могут задаваться целочисленными хешами или id (например, id пользователя соц. сети), однако нельзя сложить двух пользователей и получить третьего, исходя из их id (как это может сделать линейная модель).

Это пример категориального признака, принимающего значения из неупорядоченного конечного множества $K$. К таким признакам обычно применяют [one-hot encoding](http://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features) (вместо одного признака создают $K$ бинарных признаков - по одному на каждое возможное значение исходного признака). В sklearn это можно сделать следующим образом:

In [87]:
# для удобства работы с нашим датасетом создаем маску, указывающую на столбцы с категориальными признаками
cat_features_mask = (X_raw.dtypes == "object").values # категориальные признаки имеют тип "object"

cat_features_mask

array([False, False, False, False, False,  True,  True,  True,  True,
       False, False, False, False, False,  True,  True, False,  True,
       False, False, False, False, False, False, False, False], dtype=bool)

In [91]:
[i for i,x in enumerate(cat_features_mask) if x == True]

[5, 6, 7, 8, 14, 15, 17]

In [88]:
# кодирование категорий-строк натуральными числами
label_enc = preprocessing.LabelEncoder()
for feature in X_raw.columns[cat_features_mask]: 
    X_raw[feature] = label_enc.fit_transform(X_raw[feature])
    
X_raw.head(10) # обратите внимание на столбцы 6, 7 и прочие категориальные

,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,25,26
0,3,NaN,0,1,0,1,0,2,0,88.6,...,130,5,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,0,1,0,1,0,2,0,88.6,...,130,5,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,0,1,0,1,2,2,0,94.5,...,152,5,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,1,1,0,0,3,1,0,99.8,...,109,5,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,1,1,0,0,3,0,0,99.4,...,136,5,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
5,2,NaN,1,1,0,1,3,1,0,99.8,...,136,5,3.19,3.40,8.5,110.0,5500.0,19,25,15250.0
6,1,158.0,1,1,0,0,3,1,0,105.8,...,136,5,3.19,3.40,8.5,110.0,5500.0,19,25,17710.0
7,1,NaN,1,1,0,0,4,1,0,105.8,...,136,5,3.19,3.40,8.5,110.0,5500.0,19,25,18920.0
8,1,158.0,1,1,1,0,3,1,0,105.8,...,131,5,3.13,3.40,8.3,140.0,5500.0,17,20,23875.0
9,0,NaN,1,1,1,1,2,0,0,99.5,...,131,5,3.13,3.40,7.0,160.0,5500.0,16,22,NaN


In [90]:
# применение one-hot ecnoding

enc = preprocessing.OneHotEncoder(sparse=False)

X_cat_np = enc.fit_transform(X_raw[X_raw.columns[cat_features_mask]])
X_cat_pd = pd.DataFrame(data=X_cat_np)

In [93]:
print (X_cat_pd.shape)
X_cat_pd.head()

(203, 34)


,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


Следует заметить, что в новой матрице очень много нулевых значений. Чтобы не хранить их в памяти, можно задать параметр OneHotEncoder(sparse = True), и метод fit_transform вернет [разреженную матрицу](http://docs.scipy.org/doc/scipy/reference/sparse.html), в которой хранятся только ненулевые значения. Выполнение некоторых операций с такой матрицей может быть неэффективным, однако большинство методов sklearn умеют работать с разреженными матрицами.

Кроме того, необходимо удалить один из столбцов, созданных для каждого признака (его легко вычислить из остальных):

In [94]:
X_cat_pd.drop(enc.feature_indices_[:-1], axis=1, inplace=True)

Помимо категориальных, преобразования требуют, например, строковые признаки. Их можно превращать в матрицу частот слов [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer), матрицу частот буквосочетаний фиксированной длины, можно извлекать другие признаки (например, длина строки).

### Заполнение пропусков
В матрице объекты-признаки могут быть пропущенные значения, и это вызовет исключение при попытке передать такую матрицу в функцию обучения модели. Если пропусков немного, можно удалить объекты с пропусками из обучающей выборки. Заполнить пропуски можно разными способами:
* заполнить средними (mean, median);
* предсказывать пропущенные значения по непропущенным.

Последний вариант сложный и применяется редко. Замена пропусков средними в вещественных признаках:

In [95]:
X_real = X_raw[X_raw.columns[~cat_features_mask]]
mis_replacer = preprocessing.Imputer(strategy="mean")
X_no_mis = pd.DataFrame(data=mis_replacer.fit_transform(X_real))

In [97]:
X_no_mis.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,3.0,121.840491,0.0,1.0,0.0,88.6,168.8,64.1,48.8,2548.0,130.0,3.47,2.68,9.0,111.0,5000.0,21.0,27.0,13495.000000
1,3.0,121.840491,0.0,1.0,0.0,88.6,168.8,64.1,48.8,2548.0,130.0,3.47,2.68,9.0,111.0,5000.0,21.0,27.0,16500.000000
2,1.0,121.840491,0.0,1.0,0.0,94.5,171.2,65.5,52.4,2823.0,152.0,2.68,3.47,9.0,154.0,5000.0,19.0,26.0,16500.000000
3,2.0,164.000000,1.0,1.0,0.0,99.8,176.6,66.2,54.3,2337.0,109.0,3.19,3.40,10.0,102.0,5500.0,24.0,30.0,13950.000000
4,2.0,164.000000,1.0,1.0,0.0,99.4,176.6,66.4,54.3,2824.0,136.0,3.19,3.40,8.0,115.0,5500.0,18.0,22.0,17450.000000
5,2.0,121.840491,1.0,1.0,0.0,99.8,177.3,66.3,53.1,2507.0,136.0,3.19,3.40,8.5,110.0,5500.0,19.0,25.0,15250.000000
6,1.0,158.000000,1.0,1.0,0.0,105.8,192.7,71.4,55.7,2844.0,136.0,3.19,3.40,8.5,110.0,5500.0,19.0,25.0,17710.000000
7,1.0,121.840491,1.0,1.0,0.0,105.8,192.7,71.4,55.7,2954.0,136.0,3.19,3.40,8.5,110.0,5500.0,19.0,25.0,18920.000000
8,1.0,158.000000,1.0,1.0,1.0,105.8,192.7,71.4,55.9,3086.0,131.0,3.13,3.40,8.3,140.0,5500.0,17.0,20.0,23875.000000
9,0.0,121.840491,1.0,1.0,1.0,99.5,178.2,67.9,52.0,3053.0,131.0,3.13,3.40,7.0,160.0,5500.0,16.0,22.0,13242.613065


Всегда нужно осознавать, случайны ли пропуски в каком-то признаке. Иногда факт отсутствия информации о значении признака может сам быть важным признаком, который необходимо добавить к другим признакам.

__Пример:__ предсказание возраста пользователя по данным с его телефона. Поскольку люди старшего возраста чаще пользуются простыми телефонами, факт отсутствия каких-то данных (например, истории посещенных интернет-страниц), скорее всего, будет хорошим признаком.

Для категориальных признаков рекомендуется создавать отдельную категорию, соответствующую пропущенному значению. В наши данных пропусков в категориальных признаках нет.

In [98]:
# проверка пропусков в категориальных признаках
np.any(np.isnan(X_cat_pd))

False

In [99]:
# проверка пропусков после применения Imputer
np.any(X_no_mis.isnull())

False

### Масштабирование признаков
При начале работы с данными всегда рекомендуется приводить все признаки к одному масштабу.  Это важно по нескольким причинам:
* ускорение обучения модели (пояснение на лекции);
* улучшение численной устойчивости при работе с матрицей объекты-признаки (рядом с нулем чисел с плавающей точкой больше, чем с области больших чисел)
* для линейных моделей: интерпретация весов при признаках как меры их значимости.

Первый популярный способ масштабирования - нормализация: вычитание среднего из каждого признака и деление на стандартное отклонение. Реализация в sklearn (нормировать бинарные признаки не нужно):

In [100]:
normalizer = preprocessing.StandardScaler()
X_real_norm_np = normalizer.fit_transform(X_no_mis)
X_real_norm_pd = pd.DataFrame(data=X_real_norm_np)

Второй популярный способ: вычитание минимума из каждого признака, а затем деление на разницу максимального и минимального значения. Реализация в sklearn:

In [101]:
mm_scaler = preprocessing.MinMaxScaler()
X_mm_scaled = mm_scaler.fit_transform(X_no_mis)

Объдиняем категориальные и вещественные признаки:

In [102]:
X = pd.concat([X_real_norm_pd, X_cat_pd], axis=1)
X.columns = np.array(["f"+str(i) for i in range(X.shape[1])])

In [104]:
X.head(10)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45
0,1.734298,4.482050e-16,-1.958888,0.321342,-0.464294,-1.689532,-0.431672,-0.846292,-2.024019,-0.019023,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.734298,4.482050e-16,-1.958888,0.321342,-0.464294,-1.689532,-0.431672,-0.846292,-2.024019,-0.019023,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.130369,4.482050e-16,-1.958888,0.321342,-0.464294,-0.710457,-0.236687,-0.193602,-0.546491,0.508536,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.932333,1.329695e+00,-1.799059,0.321342,-0.464294,0.169051,0.202028,0.132743,0.233315,-0.423805,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.932333,1.329695e+00,-1.799059,0.321342,-0.464294,0.102673,0.202028,0.225984,0.233315,0.510454,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.932333,4.482050e-16,-1.799059,0.321342,-0.464294,0.169051,0.258899,0.179363,-0.259194,-0.097678,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.130369,1.140457e+00,-1.799059,0.321342,-0.464294,1.164721,1.510051,2.557018,0.807909,0.548822,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,0.130369,4.482050e-16,-1.799059,0.321342,-0.464294,1.164721,1.510051,2.557018,0.807909,0.759846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.130369,1.140457e+00,-1.799059,0.321342,2.153808,1.164721,1.510051,2.557018,0.889994,1.013075,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,-0.671596,4.482050e-16,-1.799059,0.321342,2.153808,0.119268,0.332018,0.925294,-0.710661,0.949767,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Добавление полиномиальных признаков
Генерировать все полиномиальные признаки можно следующим образом:

In [105]:
pol_feat = preprocessing.PolynomialFeatures(degree=2, include_bias=False) # не включать константный признак
X_pol = pol_feat.fit_transform(X)

In [106]:
X.shape, X_pol.shape

((203, 46), (203, 1127))

Обратите внимание, что новая матрица может занимать очень много памяти. Генерация полиномиальных признаков нужна, например, в случае, если вы хотите с помощью линейной регрессии настраивать полиномиальную модель зависимости целевого признака от данных.